In [45]:
import pandas as pd
import numpy as np
import re
import datetime

In [46]:
def convert_to_euro(value):
    numeric_part = re.findall(r'\d+\.?\d*', value)[0]
    unit = value[-1]
    if unit == 'm':
        return float(numeric_part) * 1000000
    elif unit == 'k':
        return float(numeric_part) * 1000
    else:
        return None

def convert_to_euro1(value):
    try:
        value = str(value)
        numeric_value = value[1:-1]
        if value.endswith('m'):
            multiplier = 1000000
        elif value.endswith('k'):
            multiplier = 1000
        else:
            raise ValueError("Invalid value format")
        euro_value = float(numeric_value) * multiplier
        return euro_value
    except:
        return None

def extract_number(url):
    try:
        parts = str(url).split('/')
        number = parts[-1].strip()
        return int(number)
    except:
        return None

def convert_to_year(value):
    try:
        ex_year = int(value[:2])
        if ex_year<= 25:
            year = ex_year + 2000
        else:
            year = ex_year + 1900
        return int(year)
    except:
        return None
    
def extract_number2(url):
    number = re.search(r'\d+', url)
    return int(number.group()) if number else None

def clean_int(value):
    try:
        return float(value)
    except:
        return 0

def convert_to_same_scale(value):
    value = str(value)
    if value.endswith('bn'):
        return float(value[:-2]) * 1000000000
    elif value.endswith('m'):
        return float(value[:-1]) * 1000000
    elif value.endswith('k'):
        return float(value[:-1]) * 1000
    else:
        return float(value)
    
def add_id(origin,destination,col_name):
    try:
        a = origin
        ind = a.columns.get_loc(col_name)
        main = destination
        num_blank_rows = len(a)
        blank_rows = pd.DataFrame(index=range(num_blank_rows), columns=main.columns)
        df_appended = pd.concat([main, blank_rows])
        df_appended
        for i in range(len(a)):
            df_appended.iloc[len(main) + i, 0] = a.iloc[i, ind]
        df_appended = df_appended.drop_duplicates()
        df_appended = df_appended.reset_index(drop=True)
        return df_appended
    except:
        return None

In [47]:
national = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='national')
national = national.drop_duplicates()
national = national.drop('country_url', axis=1)
national

,id,name
0,n-189,NaN
260,n-40,NaN
494,n-75,NaN
754,n-50,NaN
1014,n-157,NaN


In [48]:
competitions = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='competitions')
'''competitions['id'] = competitions['league_cups_url'].apply(lambda x: x.split('www.transfermarkt.com/')[1].split('/')[0])
competitions['country_id'] = competitions['country_url'].str.extract(r'wettbewerbe/(\d+)/saison_id')
competitions = competitions.drop(['league_cups_url','country_url'], axis=1)
competitions = competitions.drop_duplicates(subset='id')
competitions = competitions.reset_index(drop=True)
competitions['country_id'] = competitions['country_id'].astype(int)'''
competitions

,id,name,teams_number,market_value,players_number,avg_age,foreigners,country_id,league_cups_url,country_url


In [49]:
clubs = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='clubs')
clubs['value'] = clubs['value'].apply(convert_to_same_scale)
clubs = clubs.drop(['team_url','league_url'], axis=1)
clubs = clubs.drop_duplicates(subset='id')
clubs = clubs.reset_index(drop=True)
clubs

,id,name,country_id,foundation_date,value,stadium,squad_size,average_age,national_players,current_transfer_record,foreigners,season,market_value,league_id
0,c-631,Chelsea FC,n-189,NaN,1.020000e+09,Stamford Bridge,32,25.5,23,-543660000.0,21.0,NaN,NaN,NaN
1,c-281,Manchester City,n-189,NaN,1.050000e+09,Etihad Stadium,24,26.8,17,11670000.0,16.0,NaN,NaN,NaN
2,c-985,Manchester United,n-189,NaN,7.957000e+08,Old Trafford,32,26.0,18,-230030000.0,20.0,NaN,NaN,NaN
3,c-31,Liverpool FC,n-189,NaN,8.790000e+08,Anfield,30,26.7,17,-56600000.0,21.0,NaN,NaN,NaN
4,c-11,Arsenal FC,n-189,NaN,8.900000e+08,Emirates Stadium,23,25.2,17,-168560000.0,16.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,c-1108,Deportivo Alavأ©s,n-157,NaN,3.430000e+07,Mendizorroza,24,26.1,3,625000.0,10.0,NaN,NaN,NaN
173,c-1244,CD Leganأ©s,n-157,NaN,2.820000e+07,Butarque,23,28.6,3,1000000.0,8.0,NaN,NaN,NaN
174,c-12321,Girona FC,n-157,NaN,1.262000e+08,Montilivi,25,26.6,4,-15000000.0,11.0,NaN,NaN,NaN
175,c-5358,SD Huesca,n-157,NaN,2.090000e+07,El Alcoraz,24,27.5,2,1430000.0,8.0,NaN,NaN,NaN


In [50]:
players = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='players')
#players['current_value'] = players['current_value'].astype(float)
players = players.drop(['player_url','current_club_url'], axis=1)
players = players.drop_duplicates(subset='id')
players['height'] = pd.to_numeric(players['height'], errors='coerce')
players['height'] = players['height'].where(players['height'].notna(), None)
players['age'] = pd.to_numeric(players['age'], errors='coerce')
players['age'] = players['age'].where(players['age'].notna(), None)
players = players.reset_index(drop=True)
players

,id,name,birth_date,age,height,current_club_id,club_joined,contract_expires,birth_place,citizenship,position,national_id,current_value,agent,foot,shirt_number
0,p-238223,Ederson,1993-08-17 00:00:00,29.0,188.0,c-281,2017-07-01 00:00:00,2026-06-30 00:00:00,NaN,NaN,Goalkeeper,n-3439,45000000.0,Gestifute,left,NaN
1,p-85941,Stefan Ortega,1992-11-06 00:00:00,30.0,185.0,c-281,2022-07-01 00:00:00,2025-06-30 00:00:00,NaN,NaN,Goalkeeper,n-5710,6000000.0,neblung ...,right,NaN
2,p-14555,Scott Carson,1985-09-03 00:00:00,37.0,188.0,c-281,2021-07-20 00:00:00,2023-06-30 00:00:00,NaN,NaN,Goalkeeper,n-3299,250000.0,Wasserman,right,NaN
3,p-258004,Ruben Dias,1997-05-14 00:00:00,26.0,187.0,c-281,2020-09-29 00:00:00,2027-06-30 00:00:00,NaN,NaN,Defender - Centre-Back,n-3300,75000000.0,Gestifute,right,NaN
4,p-177476,Nathan Ake,1995-02-18 00:00:00,28.0,180.0,c-281,2020-08-05 00:00:00,2025-06-30 00:00:00,NaN,NaN,Defender - Centre-Back,n-3379,35000000.0,Wasserman,left,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,p-138803,Francisco Portillo,1990-06-13 00:00:00,32.0,169.0,c-3302,2021-07-22 00:00:00,2023-06-30 00:00:00,NaN,NaN,Attack - Right Winger,n-23130,1500000.0,Manuel Garcأ­a ...,right,NaN
2408,p-649016,El Bilal Toure,2001-10-03 00:00:00,21.0,NaN,c-3302,2028-06-30 00:00:00,NaN,NaN,NaN,Attack - Centre-Forward,n-3674,10000000.0,UD Almerأ­a,both,NaN
2409,p-424784,Luis Suarez,1997-12-02 00:00:00,25.0,185.0,c-3302,2023-01-01 00:00:00,2023-06-30 00:00:00,NaN,NaN,Attack - Centre-Forward,n-3816,9000000.0,CAA Stellar,right,NaN
2410,p-845568,Marko Milovanovic,2003-08-04 00:00:00,19.0,194.0,c-3302,2028-06-30 00:00:00,UD Almerأ­a B (#9),NaN,NaN,Attack - Centre-Forward,n-17383,1200000.0,UD Almerأ­a,right,NaN


In [51]:
teams_season = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='teams_season')
'''teams_season = teams_season.dropna(subset=['champion'])
teams_season['rank'] = 1
teams_season['competition_id'] = teams_season['league_cups_url'].apply(lambda x: x.split('www.transfermarkt.com/')[1].split('/')[0])
teams_season = teams_season.reset_index(drop=True)
teams_season['club_id'] = teams_season['champion']
teams_season = teams_season.drop_duplicates()
teams_season = teams_season.reset_index(drop=True)'''
teams_season['season'] = teams_season['season'].str.extract(r's-(\d{4})').astype(int)
teams_season = teams_season.drop(['league_cups_url', 'champion'], axis=1)
teams_season

,id,club_id,season,competition_id,rank,matches,win,draw,loss,goals_scored,goals_conceded,players_avg_age
0,NaN,c-985,2010,GB1,1,38,23,11,4,78,37,27.1
1,NaN,c-631,2010,GB1,2,38,21,8,9,69,33,27.1
2,NaN,c-281,2010,GB1,3,38,21,8,9,60,33,27.1
3,NaN,c-11,2010,GB1,4,38,19,11,8,72,43,27.1
4,NaN,c-148,2010,GB1,5,38,16,14,8,55,46,27.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1269,NaN,c-3709,2022,ES1,16,36,9,11,16,32,44,27.9
1270,NaN,c-2687,2022,ES1,17,36,9,11,16,28,52,27.9
1271,NaN,c-366,2022,ES1,18,36,11,5,20,33,63,27.9
1272,NaN,c-714,2022,ES1,19,36,8,11,17,47,64,27.9


In [52]:
players_season = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='players_season')
'''players_season['player_id'] = players_season['player_url_details'].apply(lambda x: x.split('/spieler/')[1].split('/')[0]).astype(int)
players_season['club_id'] = players_season['club_url'].apply(lambda x: x.split('/verein/')[1].split('/')[0])
players_season['season'] = players_season['season'].apply(convert_to_year).replace(np.nan, None).astype(int)
players_season = players_season.drop(['club_url', 'player_url_details'], axis=1)
players_season = players_season.drop_duplicates()
players_season = players_season.reset_index(drop=True)
players_season['club_id'] = players_season['club_id'].astype(int)
players_season.iloc[:, 4:18] = players_season.iloc[:, 4:18].applymap(clean_int)
players_season['Height'] = players_season['Height'].str.replace(' m', '').str.replace(',', '.').astype(float) * 100'''
players_season

,id,player_id,season,club_id,squad,appearance,point_per_goal,goals,assits,own_goals,...,minutes_per_goal,minutes_played,competition,club_url,age,position,Height,player_url_details,Current international,Agent


In [53]:
transfers = pd.read_excel('Team_Raw_Data.xlsx',sheet_name='transfers')
transfers['season'] = transfers['season'].fillna('').replace('', None).astype(int)
transfers = transfers.drop(['player_url','signed_from_url','team_url'], axis=1)
transfers = transfers.drop_duplicates()
transfers = transfers.reset_index(drop=True)
transfers

,id,player_id,origin_club_id,destination_club_id,season,market_value,fee
0,NaN,p-37367,c-12181,c-123,2023,50000.0,NaN
1,NaN,p-37367,c-21322,c-12181,2022,50000.0,NaN
2,NaN,p-37367,c-7971,c-21322,2021,25000.0,NaN
3,NaN,p-37367,c-648,c-7971,2017,300000.0,NaN
4,NaN,p-37367,c-993,c-648,2016,300000.0,NaN
...,...,...,...,...,...,...,...
30614,NaN,p-41866,c-42,c-15,2011,1000000.0,NaN
30615,NaN,p-41866,c-15,c-42,2009,1000000.0,NaN
30616,NaN,p-41866,c-8715,c-15,2008,1000000.0,400000.0
30617,NaN,p-41866,c-2038,c-8715,2007,NaN,1000000.0


In [54]:
national = add_id(players, national, 'national_id')
clubs = add_id(players, clubs, 'current_club_id')
clubs = add_id(teams_season, clubs, 'club_id')
competitions = add_id(teams_season, competitions, 'competition_id')
players = add_id(transfers, players, 'player_id')
clubs = add_id(transfers, clubs, 'origin_club_id')
clubs = add_id(transfers, clubs, 'destination_club_id')

In [ ]:
national = national.drop_duplicates(subset=['id'])
national = national.dropna(subset=['id'])
national = national.reset_index(drop=True)
players = players.drop_duplicates(subset=['id'])
players = players.reset_index(drop=True)
players = players.dropna(subset=['id'])
clubs = clubs.drop_duplicates(subset=['id'])
clubs = clubs.dropna(subset=['id'])
clubs = clubs.reset_index(drop=True)
competitions = competitions.drop_duplicates(subset=['id'])
competitions = competitions.dropna(subset=['id'])
competitions = competitions.reset_index(drop=True)

In [ ]:
national.to_csv('national.csv', index=False)
competitions.to_csv('competitions.csv', index=False)
clubs.to_csv('clubs.csv', index=False)
players.to_csv('players.csv', index=False)
teams_season.to_csv('teams_season.csv', index=False)
players_season.to_csv('players_season.csv', index=False)
transfers.to_csv('transfers.csv', index=False)

In [ ]:
writer = pd.ExcelWriter(r'E:\Quera\Projects\TransferMarket\Database\Team_Data.xlsx')

national.to_excel(writer, sheet_name='national', index=False)
competitions.to_excel(writer, sheet_name='competitions', index=False)
clubs.to_excel(writer, sheet_name='clubs', index=False)
players.to_excel(writer, sheet_name='players', index=False)
teams_season.to_excel(writer, sheet_name='teams_season', index=False)
players_season.to_excel(writer, sheet_name='players_season', index=False)
transfers.to_excel(writer, sheet_name='transfers', index=False)

writer.save()